# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data/cities.csv') 
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lorengau,-2.0226,147.2712,83.32,79.0,18.0,11.16,PG,1.628224e+09
1,rikitea,-23.1203,-134.9692,74.16,75.0,10.0,14.50,PF,1.628224e+09
2,nanortalik,60.1432,-45.2371,45.88,96.0,100.0,2.06,GL,1.628224e+09
3,hithadhoo,-0.6000,73.0833,82.87,69.0,97.0,7.58,MV,1.628223e+09
4,avarua,-21.2078,-159.7750,77.05,57.0,75.0,2.30,CK,1.628223e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
!pip3 install gmaps
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [4]:
# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
weather_df.head()
locations = weather_df[["Lat", "Lng"]]
locations


# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

# Add Heatmap layer to map.
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 90 degrees but higher than 75.
# Wind speed less than 10 mph.
# Zero cloudiness.

# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
# Also added and updated temperature values above:
# Minimal humidity (eg <= 40) 

# Drop any rows that don’t contain all three conditions. You want to be sure the weather is ideal.
ideal_weather_df = weather_df.loc[(weather_df['Max Temp'] >= 75) 
               & (weather_df['Max Temp'] <= 90) 
               & (weather_df['Wind Speed'] <= 10) 
               & (weather_df['Cloudiness'] == 0)
               & (weather_df['Humidity'] <= 40)
            ]
ideal_weather_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,ajdabiya,30.7554,20.2263,84.36,30.0,0.0,7.31,LY,1.628224e+09
167,urumqi,43.8010,87.6005,83.35,23.0,0.0,6.71,CN,1.628224e+09
201,turayf,31.6725,38.6637,77.25,36.0,0.0,0.00,SA,1.628224e+09
247,jalu,29.0331,21.5482,84.96,20.0,0.0,8.77,LY,1.628224e+09
334,nyurba,63.2842,118.3319,77.43,24.0,0.0,9.44,RU,1.628224e+09
436,tukrah,32.5341,20.5791,86.38,34.0,0.0,0.72,LY,1.628224e+09
457,birjand,32.8663,59.2211,79.27,15.0,0.0,6.91,IR,1.628225e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [7]:
import json

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

hotel_df = ideal_weather_df.copy()

# Use the lat/lng for ideal weathed cities
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotels = requests.get(base_url, params=params)

#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotels = hotels.json()
#     print(json.dumps(hotels, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# Visualize to confirm hotel data appears
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,ajdabiya,30.7554,20.2263,84.36,30.0,0.0,7.31,LY,1.628224e+09,Almadina Hotel
167,urumqi,43.8010,87.6005,83.35,23.0,0.0,6.71,CN,1.628224e+09,Sheraton Urumqi Hotel
201,turayf,31.6725,38.6637,77.25,36.0,0.0,0.00,SA,1.628224e+09,Golden Dune Hotel
247,jalu,29.0331,21.5482,84.96,20.0,0.0,8.77,LY,1.628224e+09,Jalu Hotel
334,nyurba,63.2842,118.3319,77.43,24.0,0.0,9.44,RU,1.628224e+09,Gostinitsa Alrosa
436,tukrah,32.5341,20.5791,86.38,34.0,0.0,0.72,LY,1.628224e+09,مصيف مرحبا للعائلات
457,birjand,32.8663,59.2211,79.27,15.0,0.0,6.91,IR,1.628225e+09,هتل پرويز


In [8]:
#drop NaN named hotels optionally
hotel_df = hotel_df.loc[hotel_df['Hotel Name'].notna()]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,ajdabiya,30.7554,20.2263,84.36,30.0,0.0,7.31,LY,1.628224e+09,Almadina Hotel
167,urumqi,43.8010,87.6005,83.35,23.0,0.0,6.71,CN,1.628224e+09,Sheraton Urumqi Hotel
201,turayf,31.6725,38.6637,77.25,36.0,0.0,0.00,SA,1.628224e+09,Golden Dune Hotel
247,jalu,29.0331,21.5482,84.96,20.0,0.0,8.77,LY,1.628224e+09,Jalu Hotel
334,nyurba,63.2842,118.3319,77.43,24.0,0.0,9.44,RU,1.628224e+09,Gostinitsa Alrosa
436,tukrah,32.5341,20.5791,86.38,34.0,0.0,0.72,LY,1.628224e+09,مصيف مرحبا للعائلات
457,birjand,32.8663,59.2211,79.27,15.0,0.0,6.91,IR,1.628225e+09,هتل پرويز


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add hotel layer ontop of heat map

# Create hotel layer
# hotel_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content=hotel_info
# )
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig = gmaps.figure()
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))